In [2]:
import cv2
import mediapipe as mp
import numpy as np
from keras.models import load_model
from keras.preprocessing.image import img_to_array


In [3]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

In [4]:
# Defining the hand positions
def hand_possition_class(landmarks):  
    # giving names to a few landmarks for readability
    thumb_tip = landmarks[mp_hands.HandLandmark.THUMB_TIP]
    index_tip = landmarks[mp_hands.HandLandmark.INDEX_FINGER_TIP]
    middle_tip = landmarks[mp_hands.HandLandmark.MIDDLE_FINGER_TIP]
    ring_tip = landmarks[mp_hands.HandLandmark.RING_FINGER_TIP]
    pinky_tip = landmarks[mp_hands.HandLandmark.PINKY_TIP]
    
    thumb_mcp = landmarks[mp_hands.HandLandmark.THUMB_MCP]
    index_mcp = landmarks[mp_hands.HandLandmark.INDEX_FINGER_MCP]
    middle_mcp = landmarks[mp_hands.HandLandmark.MIDDLE_FINGER_MCP]
    ring_mcp = landmarks[mp_hands.HandLandmark.RING_FINGER_MCP]
    pinky_mcp = landmarks[mp_hands.HandLandmark.PINKY_MCP]
   
     
  # Calculate distances between tip and mcp
    dist_thumb_index_tip = np.linalg.norm(
        np.array([thumb_tip.x, thumb_tip.y]) - 
        np.array([index_tip.x, index_tip.y])
        )
    
    dist_index_mcp_tip = np.linalg.norm(
        np.array([index_tip.x, index_tip.y]) - 
        np.array([index_mcp.x, index_mcp.y])
        )
    
    dist_middle_mcp_tip = np.linalg.norm(
        np.array([middle_tip.x, middle_tip.y]) - 
        np.array([middle_mcp.x, middle_mcp.y])
        )
    
    dist_ring_mcp_tip = np.linalg.norm(
        np.array([ring_tip.x, ring_tip.y]) - 
        np.array([ring_mcp.x, ring_mcp.y])
        )
    
    dist_pinky_mcp_tip = np.linalg.norm(
        np.array([pinky_tip.x, pinky_tip.y]) - 
        np.array([pinky_mcp.x, pinky_mcp.y])
        )
    
    
 # Classification of rock papper scissors
    if (dist_index_mcp_tip < 0.1 and 
        dist_middle_mcp_tip < 0.1 and 
        dist_ring_mcp_tip < 0.1 and 
        dist_pinky_mcp_tip < 0.1):
        return "Rock"
    elif (dist_index_mcp_tip > 0.1 and 
          dist_middle_mcp_tip > 0.1 and 
          dist_ring_mcp_tip > 0.1 and 
          dist_pinky_mcp_tip > 0.1):
        return "Paper"
    else:
        return "Scissors"

In [5]:
# Game rules
def determine_winner(hand1, hand2):
    if hand1 == hand2:
        return "Tie"
    if ((hand1 == "Rock" and hand2 == "Scissors") or 
        (hand1 == "Scissors" and hand2 == "Paper") or 
        (hand1 == "Paper" and hand2 == "Rock")):
        return "Player 1 Wins!"
    return "Player 2 Wins!"


In [10]:
cap = cv2.VideoCapture(0)

# Detect face
face_classifier = cv2.CascadeClassifier(r'C:\Users\Hanss\Documents\Utbildning\Deep Learning\Emotion_Detection_CNN\haarcascade_frontalface_default.xml')
emotion_classifier = load_model(r"C:\Users\Hanss\Documents\Utbildning\Deep Learning\emotion_model.keras")

# Detect and track hands, must be 50% sure that it is a hand 
# and 50% sure it is tracking the same hand. 
with mp_hands.Hands(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Skipped frame")
            continue

        # Gives a mirror image for the user, easier for user to understand
        image = cv2.flip(image, 1)

        # Open CV uses BGR color format changes to RGB color 
        # format that mediapipe uses.
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        
        results = hands.process(image_rgb)
        
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray)
        
        image.flags.writeable = True
        detected_hand_list = []

        # Enhancing the visual understanding of hand positions in the processed image.
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                
                hand_posistion = hand_possition_class(hand_landmarks.landmark)
                detected_hand_list.append(hand_posistion)

                # Displaying the classification
                cv2.putText(image, hand_posistion, 
                            (int(hand_landmarks.landmark[0].x * image.shape[1] + 10), 
                             int(hand_landmarks.landmark[0].y * image.shape[0])),
                            cv2.FONT_HERSHEY_TRIPLEX, 0.8, (128, 0, 128), 1, cv2.LINE_4
                            )
        # Displaying the winner
        if len(detected_hand_list) == 2:
            outcome = determine_winner(detected_hand_list[0], detected_hand_list[1])
            middle_x = (image.shape[1] // 2) - (14 // 2)
            cv2.putText(image, outcome, ((middle_x, 50)),
                        cv2.FONT_HERSHEY_TRIPLEX, 0.8, (255, 0, 0), 1, cv2.LINE_8)

    
        cv2.imshow('Rock Paper Scissors', image)

        if cv2.waitKey(5) & 0xFF == 27:
            break

cap.release()
cv2.destroyAllWindows()